In [10]:
import pandas as pd
import numpy as np

In [2]:
path_corpus = "/Users/philippmetzger/OneDrive/PHILIPP/NOVA IMS/2nd Semester/06 Text Mining 4 ECTS/00 Project/corpus/"
cs_en = pd.read_csv(path_corpus + "cs-en/scores.csv")
de_en = pd.read_csv(path_corpus + "de-en/scores.csv")
en_fi = pd.read_csv(path_corpus + "en-fi/scores.csv")
en_zh = pd.read_csv(path_corpus + "en-zh/scores.csv")
ru_en = pd.read_csv(path_corpus + "ru-en/scores.csv")
zh_en = pd.read_csv(path_corpus + "zh-en/scores.csv")

In [11]:
descriptions = ["Russian into English", "German into English", "Czech into English", "Chinese into English", "English into Chinese", "English into Finish"]

rows = []
zscores = []
avgscores = []
annots = []

i = 0

for element in [ru_en, de_en, cs_en, zh_en, en_zh, en_fi]:
    rows.append(element.shape[0])
    zscores.append(np.round(element["z-score"].mean(),2))
    avgscores.append(np.round(element["avg-score"].mean(), 2))
    annots.append(np.round(element["annotators"].mean(),2))
    i += 1                   
    
exploration_df = pd.DataFrame([rows, zscores, avgscores, annots]).T.rename(columns={0:"rows", 1:"avg z-score", 2:"avg avg-score", 3:"avg annotators"})
exploration_df["description"] = descriptions
exploration_df = exploration_df.set_index("description")
exploration_df

,rows,avg z-score,avg avg-score,avg annotators
description,,,,
Russian into English,17980.0,0.01,74.50,1.30
German into English,21704.0,0.00,71.85,1.50
Czech into English,11585.0,-0.03,69.24,1.89
Chinese into English,26419.0,-0.05,66.06,1.42
English into Chinese,10221.0,-0.06,65.98,1.58
English into Finish,6748.0,-0.14,45.12,1.23


In [13]:
de_en.head(3)

,source,reference,translation,z-score,avg-score,annotators
0,"Ihr Zeitlupentempo maßen sie, als sie vor Spit...",Her timeless pace measures them when they equi...,Their slow speed was measured by researchers o...,-0.345024,76.0,1
1,"Er sagte, dass die Bereiche ruhige Treffpunkte...",He said the areas offer quiet meeting points b...,He said the spaces provided calm meeting point...,0.903800,97.5,2
2,Für die Geschäftsleute an der B 27 ist es nur ...,"For businessmen at the B 27, it's only a small...",This is only a small consolation for businesse...,0.700503,94.0,1


# Word Mover's distance
## https://towardsdatascience.com/word-movers-distance-for-text-similarity-7492aeca71b0

In [103]:
from nltk.corpus import stopwords
from nltk import download
import re
import torch
import torchtext.vocab as vocab
from tqdm import tqdm_notebook as tqdm

In [92]:
glove = vocab.GloVe(name='6B', dim=50)
print('Loaded {} words'.format(len(glove.itos)))

.vector_cache/glove.6B.zip: 862MB [03:26, 4.18MB/s]                               
100%|█████████▉| 399999/400000 [00:09<00:00, 44101.47it/s]


Loaded 400000 words


In [93]:
def get_word(word):
    return glove.vectors[glove.stoi[word]]

In [95]:
row = 0

In [98]:
a = de_en.iloc[row,1]
b = de_en.iloc[row,2]
print(a)
print(b)

a = a.lower()
a = re.sub("[^a-z]", ' ', a)
a = a.split()

b = b.lower()
b = re.sub("[^a-z]", ' ', b)
b = b.split()

a = [word for word in a if word not in stop]
b = [word for word in b if word not in stop]
print(a)
print(b)

Her timeless pace measures them when they equipped six animals with a broadcaster before Spitsbergen.
Their slow speed was measured by researchers off Svalbard, who fitted six animals with a tracker.
['timeless', 'pace', 'measures', 'equipped', 'six', 'animals', 'broadcaster', 'spitsbergen']
['slow', 'speed', 'measured', 'researchers', 'svalbard', 'fitted', 'six', 'animals', 'tracker']


In [99]:
download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philippmetzger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [100]:
stop = stopwords.words('english')

In [ ]:
dist = 0
for word_a in sentence_a:
    distances_this = []
    for word_b in sentence_b:
        distances_this.append(torch.dist(get_word(word_a), get_word(word_b)))
    min_dist = min(distances_this)
    print(min_dist)
    dist += min_dist
print(dist)

In [108]:
distances = []

for i in tqdm(range(de_en.shape[0])):
    
    a = de_en.iloc[i,1]
    b = de_en.iloc[i,2]

    a = a.lower()
    a = re.sub("[^a-z]", ' ', a)
    a = a.split()

    b = b.lower()
    b = re.sub("[^a-z]", ' ', b)
    b = b.split()

    a = [word for word in a if word not in stop]
    b = [word for word in b if word not in stop]
    
    dist = 0
    for word_a in a:
        distances_this = []
        for word_b in b:
            
            try:
                distances_this.append(torch.dist(get_word(word_a), get_word(word_b)))
            except:
                distances_this.append(99999999999)
        
        try:
            min_dist = min(distances_this)
        except:
            print(i)
        
        dist += min_dist
    
    distances.append(dist)
distances

<ipython-input-108-d6517ab23d2b>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(de_en.shape[0])):


  0%|          | 0/21704 [00:00<?, ?it/s]

772
1492
1492
1492
1492
1492
1492
2232
3302
3302
3302
3302
3302
3302
6298
6298
6298
6298
6298
6298
8094
8159
8159
8159
8159
8159
9937
9937
9937
9937
9937
10078
11757
11757
11757
11757
11757
11757
13866
13866
13866
13866
16498
19651
21186


[tensor(24.6994),
 tensor(10.4937),
 tensor(1.0000e+11),
 tensor(13.4154),
 tensor(8.2626),
 tensor(6.9412),
 tensor(1.0000e+11),
 tensor(9.6643),
 tensor(2.0000e+11),
 tensor(5.5232),
 tensor(6.2254),
 tensor(0.),
 tensor(3.2087),
 tensor(14.7071),
 tensor(12.0196),
 tensor(8.3754),
 tensor(12.7141),
 tensor(3.7852),
 tensor(24.2575),
 tensor(25.6086),
 tensor(12.7548),
 tensor(1.0000e+11),
 tensor(16.0747),
 tensor(15.7955),
 tensor(8.2538),
 tensor(1.0000e+11),
 tensor(13.5246),
 tensor(43.3557),
 tensor(1.0000e+11),
 tensor(16.3947),
 tensor(0.),
 tensor(1.0000e+11),
 tensor(2.0000e+11),
 tensor(3.5852),
 tensor(2.7113),
 tensor(17.4682),
 tensor(2.0000e+11),
 tensor(1.0000e+11),
 tensor(9.8887),
 tensor(13.8396),
 tensor(1.0000e+11),
 tensor(2.0000e+11),
 tensor(14.8549),
 tensor(11.0227),
 tensor(23.9331),
 tensor(16.8168),
 tensor(15.3569),
 tensor(15.4084),
 tensor(23.6512),
 tensor(15.9953),
 tensor(23.2779),
 tensor(1.0000e+11),
 tensor(4.2416),
 tensor(13.6188),
 tensor(12.6

In [109]:
de_en.iloc[772,1]

"What's supposed to be?"

In [110]:
de_en.iloc[772,2]

'So what?'

In [113]:
"so" in stop

True

In [114]:
"what" in stop

True